## <center>Natural Language Processing (Tugas 2)</center>
<strong> NIM  : 1301178418 </strong><br>
<strong> Nama : Bastomy </strong> <br>
<strong> Kelas : ICM-39-GAB </strong>

#### Deskripsi Tugas

Deskripsi
Buatlah program yang mengimplementasikan dua pendekatan POS-tagging. Sebagai input adalah
kalimat Bahasa Indonesia and output adalah kalimat setelah setiap katanya diberi tag yang tepat.
Kedua pendekatan yang harus diimplementasikan adalah:
1. Baseline (menggunakan POS Tag yang paling sering muncul)
2. HMM.

### Import library

In [39]:
import pandas as pd
import re
import operator

<strong>pandas</strong> =  untuk mempermudah membuat array menjadi data tabel <br>
<strong>re</strong> = regular expression untuk membagi string menjadi array berdasarkan tab (\t) <br>
<strong>operator</strong> = untuk mengambil item

### Fungsi untuk membaca data training dan testing menjadi array

In [ ]:
def read_data(train):
    dt_train = {'kata':[], 'tag':[]}
    for line in open(train):
        if not line.startswith('<kalimat') and not line.startswith('</kalimat'):
            baris = re.split(r'\t+', line.rstrip('\t'))
            dt_train['kata'].append(baris[0].lower())
            dt_train['tag'].append(baris[1].replace('\n',''))
    data = pd.DataFrame(dt_train)        
    return data

read_data adalah fungsi untuk membaca data .txt menjadi sebuah array <br>
terdapat dua kolom array yaitu kata dan tag  <br>
setiap baris data akan di looping dan di cek jika tidak berawalan <kalimat dan </kalimat maka akan lanjut ke tahap berikutnya yaitu membagi setiap baris menjadi array berdasarkan \t (Tab) <br>
kemudian akan di simpan ke dt_train dan di set menjadi Dataframe untuk di return

### Fungsi untuk menghitung jumlah tag berdasarkan kata

In [ ]:
def post_tag_frequency(kata, data_training):
    hitungTag = data_training.loc[data_training['kata'] == kata]['tag'].value_counts()
    if(len(hitungTag) == 0):
        return 'NN','0'
    tagTerbaik = max(hitungTag.items(),key=operator.itemgetter(1))
    return tagTerbaik[0],tagTerbaik[1]

Menghitung jumlah tag berdasarkan kata  <br>
variabel hitungTag akan menyimpan jumlah tag berdasarkan kata  <br>
jika tag bernilai 0 maka akan di terun tag NN dengan jumlah 0  (untuk menghandle kata yg tidak terdaftar)<br>
kemudian akan dicari max dari tag yg tersedia  <br>
fungsi akan mengembalikan Tag dan jumlah tag nya <br>
contoh (PRP   44) <br>

### Fungsi prediksi tag BASELINE

In [ ]:
def baseline(data_testing,data_training):
    index = 0 
    tag_correct = 0
    result = {'kata':[], 'prediksi':[], 'tag':[]}
    akurasi = 0
    for i in data_testing['kata']:
        tag,count = post_tag_frequency(i, data_training)
        if(tag == data_testing['tag'][index]):
            tag_correct+=1
        result['kata'].append(i)
        result['prediksi'].append(tag)
        result['tag'].append(data_testing['tag'][index])
        index+=1
    data = pd.DataFrame(result)
    akurasi = (tag_correct/index)*100
    return data,akurasi

Baseline akan memprediksi Tag berdasarkan kemunculan terbanyak <br>
cara kerja fungsi diatas adalah melooping semua kata beserta Tag <br>
kemudian kata tersebut akan diprediksi Tag nya berdasarkan data training <br>
post_tag_frequency akan mengembalikan tag beserta jumlahnya kemunculannya, tag dari post_tag_frequency akan dijadikan prediksi <br>
akurasi dihitung dari jumlah tag yg benar dibagi dengan total seluruh kata

### Read Data

In [ ]:
dt_training = read_data('corpus/training.txt')
dt_testing = read_data('corpus/testing.txt')

simpan data training dan testing kedalam variabel

### NO 1. Baseline (menggunakan POS Tag yang paling sering muncul)

In [ ]:
data, akurasi = baseline(dt_testing,dt_training)

Data testing dan prediksi TAG

In [ ]:
data

Akurasi Tag (persen)

In [ ]:
akurasi

### No 2 HMM

### Import library

In [7]:
from nltk import word_tokenize
from sklearn import tree
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import Pipeline
import time

import library yang akan digunakan seperti :<br>
<strong>word_tokenize</strong> : untuk membuat kalimat menjadi token<br>
<strong>tree</strong> : untuk membuat DecisionTreeClassifier<br>
<strong>DictVectorizer</strong> : untuk merubah feature menjadi matrix vektor<br>
<strong>time</strong> : untuk menghitung lama program di eksekusi<br>

### Fungsi untuk mengekstrak data menjadi kalimat + tag

In [2]:
def read_data_sentences(data):
    sentences = []
    tags = []
    with open(data) as f:
        content = f.readlines()
    contents = [x.strip() for x in content]
    sentence = []
    tag = []
    for i in contents:
        if not i.startswith('<kalimat') and not i.startswith('</kalimat'):
            content_part = i.split('\t')
            if (len(content_part)==2):
                sentence.append(content_part[0])
                tag.append(content_part[1])
        elif i.startswith('</kalimat'):
            sentences.append(sentence)
            tags.append(tag)
            sentence = []
            tag = []
    return sentences,tags    

<strong>Contoh:</strong> <br>
<table style="border: 1px solid black;">
	<tr style="border: 1px solid black;">
		<td style="border: 1px solid black;" style="border: 1px solid black;">Kalimat</td>
		<td style="border: 1px solid black;">namun</td>
		<td style="border: 1px solid black;">laba</td>
		<td style="border: 1px solid black;">usaha</td>
		<td style="border: 1px solid black;">dalam</td>
		<td style="border: 1px solid black;">periode</td>
		<td style="border: 1px solid black;">tersebut</td>
		<td style="border: 1px solid black;">menyusut</td>
		<td style="border: 1px solid black;">dari</td>
		<td style="border: 1px solid black;">semula</td>
		<td style="border: 1px solid black;">Rp</td>
		<td style="border: 1px solid black;">58,27</td>
		<td style="border: 1px solid black;">miliar</td>
		<td style="border: 1px solid black;">menjadi</td>
		<td style="border: 1px solid black;">Rp</td>
		<td style="border: 1px solid black;">41,39</td>
		<td style="border: 1px solid black;">miliar</td>
		<td style="border: 1px solid black;">.</td>
	</tr>
	<tr style="border: 1px solid black;">
		<td style="border: 1px solid black;" style="border: 1px solid black;">Tag</td>
		<td style="border: 1px solid black;">CC</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">IN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">PR</td>
		<td style="border: 1px solid black;">VB</td>
		<td style="border: 1px solid black;">IN</td>
		<td style="border: 1px solid black;">NN</td>
		<td style="border: 1px solid black;">SYM</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">VB</td>
		<td style="border: 1px solid black;">SYM</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">CD</td>
		<td style="border: 1px solid black;">Z</td>
	</tr>
</table>

In [3]:
def extract_features(sentence, index):
    return {
        'word': sentence[index],
        'prefix-1': sentence[index][0],
        'prefix-2': sentence[index][:2],
        'prefix-3': sentence[index][:3],
        'suffix-1': sentence[index][-1],
        'suffix-2': sentence[index][-2:],
        'suffix-3': sentence[index][-3:],
        'prev_word': '' if index == 0 else sentence[index - 1],
        'next_word': '' if index == len(sentence) - 1 else sentence[index + 1],
    }

In [4]:
def transform_to_dataset(sentences, tags):
    X, y = [], []
    for sentence_idx in range(len(sentences)):
        for index in range(len(sentences[sentence_idx])):
            X.append(extract_features(sentences[sentence_idx], index))
            y.append(tags[sentence_idx][index])
    return X, y

In [5]:
training_sentences,training_tags = read_data_sentences('corpus/training.txt')
testing_sentences,testing_tags = read_data_sentences('corpus/testing.txt')

In [6]:
X, y = transform_to_dataset(training_sentences, training_tags)

In [8]:
start = time.time()
clf = Pipeline([
    ('vectorizer', DictVectorizer(sparse=False)),
    ('classifier', tree.DecisionTreeClassifier(criterion='entropy'))
])
clf.fit(X, y)
finish = time.time()
print('Training completed in ',finish-start,' seconds')

Training completed in  396.23731422424316  seconds


In [10]:
X_test, y_test = transform_to_dataset(testing_sentences, testing_tags)
 
print("Accuracy:")
print(clf.score(X_test, y_test)*100,'persen')

Accuracy:
94.98997995991984 persen


In [43]:
# Test model yang sudah dilatih dengan kalimat masukan bebas
from nltk import word_tokenize

def pos_tag(sentence):
    tags = clf.predict([extract_features(sentence, index) for index in range(len(sentence))])
    data = pd.DataFrame({"kata":sentence,'tag':tags})
    return data


In [45]:
result = pos_tag(word_tokenize('Pemerintah Kabupaten Serang memberikan insentif kepada 1.796 Kader Pos Keluarga Berencana dan Kader Subpos KB. Bupati Serang Ratu Tatu Chasanah mengatakan hal tersebut sebagai apresiasi kader yang terus berupaya mengendalikan laju pertumbuhan penduduk di Kabupaten Serang'))

In [46]:
result

,kata,tag
0,Pemerintah,NN
1,Kabupaten,NNP
2,Serang,NNP
3,memberikan,VB
4,insentif,JJ
5,kepada,IN
6,1.796,CD
7,Kader,NNP
8,Pos,NNP
9,Keluarga,NNP


In [56]:
def read_file_init_table(fname):
    tag_count = {}
    tag_count['<start>'] = 0
    word_tag = {}
    tag_trans = {}
    with open(fname) as f:
        content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]
    idx_line = 0
    is_first_word = 0
    
    while idx_line < len(content):
        prev_tag = '<start>'
        while not content[idx_line].startswith('</kalimat'):
            if  not content[idx_line].startswith('<kalimat'):
                content_part = content[idx_line].split('\t')
                if content_part[1] in tag_count:
                    tag_count[content_part[1]] += 1
                else:
                    tag_count[content_part[1]] = 1
                    
                current_word_tag = content_part[0]+','+content_part[1]
                if current_word_tag in word_tag:
                    word_tag[current_word_tag] += 1
                else:    
                    word_tag[current_word_tag] = 1
                    
                if is_first_word == 1:
                    current_tag_trans = '<start>,'+content_part[1]
                    is_first_word = 0
                else:
                    current_tag_trans = prev_tag+','+content_part[1]
                    
                if current_tag_trans in tag_trans:
                    tag_trans[current_tag_trans] += 1
                else:
                    tag_trans[current_tag_trans] = 1                    
                prev_tag = content_part[1]   
                
            else:
                tag_count['<start>'] += 1
                is_first_word = 1
            idx_line = idx_line + 1

        idx_line = idx_line+1       
    return tag_count, word_tag, tag_trans

In [106]:
tag_count, word_tag, tag_trans = read_file_init_table('corpus/training.txt')

In [130]:
word_tag

{'Kera,NN': 1,
 'untuk,SC': 126,
 'amankan,VB': 1,
 'pesta olahraga,NN': 1,
 'Pemerintah,NNP': 2,
 'kota,NNP': 3,
 'Delhi,NNP': 5,
 'mengerahkan,VB': 1,
 'monyet,NN': 12,
 'mengusir,VB': 4,
 'monyet-monyet,NN': 2,
 'lain,JJ': 29,
 'yang,SC': 538,
 'berbadan,VB': 2,
 'lebih,RB': 80,
 'kecil,JJ': 14,
 'dari,IN': 251,
 'arena,NN': 2,
 'Pesta Olahraga,NNP': 1,
 'Persemakmuran,NNP': 1,
 '.,Z': 1012,
 'Beberapa,CD': 1,
 'laporan,NN': 15,
 'menyebutkan,VB': 10,
 'setidaknya,RB': 2,
 '10,CD': 21,
 'ditempatkan,VB': 2,
 'di,IN': 360,
 'luar,NN': 5,
 'lomba,NN': 1,
 'dan,CC': 481,
 'pertandingan,NN': 1,
 'ibukota,NNP': 2,
 'India,NNP': 6,
 'Pemkot,NNP': 1,
 'memiliki,VB': 20,
 '28,CD': 6,
 'berencana,VB': 9,
 'mendatangkan,VB': 2,
 'sejenis,NN': 1,
 'negara bagian,NNP': 1,
 'Rajasthan,NNP': 1,
 'Jumlah,NN': 5,
 'mencapai,VB': 97,
 'ribuan,CD': 1,
 '",",Z': 1393,
 'sebagian besar,NN': 3,
 'berada,VB': 14,
 'kantor-kantor,NN': 1,
 'pemerintah,NN': 52,
 'hewan,NN': 1,
 'ini,PR': 199,
 'dianggap,VB'

In [58]:
def create_trans_prob_table(tag_trans, tag_count):
    print(tag_trans)
    trans_prob = {}
    for tag1 in tag_count.keys():
        for tag2 in tag_count.keys():
            #print('tag1 = ')
            #print(tag1)
            trans_idx = tag1+','+tag2
            #print('trans_idx = ')
            #print(trans_idx)
            if trans_idx in tag_trans:
                #print(trans_idx)
                trans_prob[trans_idx] = tag_trans[trans_idx]/tag_count[tag1]
    return trans_prob

In [59]:
trans_prob = create_trans_prob_table(tag_trans, tag_count)
print(trans_prob)

{'<start>,NN': 321, 'NN,SC': 437, 'SC,VB': 496, 'VB,NN': 938, '<start>,NNP': 169, 'NNP,NNP': 1959, 'NNP,VB': 272, 'NN,JJ': 494, 'JJ,SC': 90, 'VB,RB': 65, 'RB,JJ': 137, 'JJ,IN': 154, 'IN,NN': 1280, 'NN,NNP': 509, 'NNP,Z': 743, '<start>,CD': 26, 'CD,NN': 432, 'NN,VB': 296, 'RB,CD': 40, 'VB,IN': 479, 'NN,NN': 2103, 'NN,CC': 238, 'CC,NN': 334, 'NN,IN': 533, 'IN,NNP': 440, 'VB,CD': 253, 'CC,VB': 86, 'VB,VB': 140, 'CD,Z': 434, 'Z,NN': 386, 'NN,PR': 324, 'PR,VB': 140, 'JJ,Z': 204, 'VB,JJ': 80, 'Z,VB': 331, 'Z,CC': 127, 'JJ,CC': 66, 'VB,PRP': 174, 'PRP,MD': 45, 'MD,VB': 408, 'SC,NN': 206, 'JJ,JJ': 29, 'JJ,VB': 74, 'VB,Z': 189, 'CC,FW': 19, 'FW,VB': 22, 'SC,RB': 74, 'RB,VB': 171, 'NN,Z': 504, 'JJ,MD': 20, 'IN,CD': 239, '<start>,Z': 26, 'PR,SC': 17, 'Z,Z': 54, 'VB,NNP': 158, 'NNP,IN': 261, '<start>,IN': 182, 'CC,RB': 37, '<start>,NEG': 1, 'NEG,JJ': 19, 'JJ,PR': 22, 'PR,NN': 31, 'PR,NEG': 15, 'NEG,MD': 22, 'VB,SC': 137, 'PRP,JJ': 11, '<start>,CC': 88, 'CC,MD': 17, 'Z,NNP': 338, 'NNP,RB': 40, 'SC,

In [126]:
def create_emission_prob_table(word_tag, tag_count):
    emission_prob = {}
    for word_tag_entry in word_tag.keys():
        word_tag_split = word_tag_entry.split(',')
        current_word = word_tag_split[0]
        current_tag = word_tag_split[1]
        if (len(word_tag_split)!=3):
            emission_key = current_word+','+current_tag
            print(word_tag_entry)
            emission_prob[emission_key] = word_tag[word_tag_entry]/tag_count[current_tag] 
    return emission_prob

In [127]:
emission_prob = create_emission_prob_table(word_tag, tag_count)
print(emission_prob)

Kera,NN
untuk,SC
amankan,VB
pesta olahraga,NN
Pemerintah,NNP
kota,NNP
Delhi,NNP
mengerahkan,VB
monyet,NN
mengusir,VB
monyet-monyet,NN
lain,JJ
yang,SC
berbadan,VB
lebih,RB
kecil,JJ
dari,IN
arena,NN
Pesta Olahraga,NNP
Persemakmuran,NNP
.,Z
Beberapa,CD
laporan,NN
menyebutkan,VB
setidaknya,RB
10,CD
ditempatkan,VB
di,IN
luar,NN
lomba,NN
dan,CC
pertandingan,NN
ibukota,NNP
India,NNP
Pemkot,NNP
memiliki,VB
28,CD
berencana,VB
mendatangkan,VB
sejenis,NN
negara bagian,NNP
Rajasthan,NNP
Jumlah,NN
mencapai,VB
ribuan,CD
sebagian besar,NN
berada,VB
kantor-kantor,NN
pemerintah,NN
hewan,NN
ini,PR
dianggap,VB
mengganggu,VB
ketertiban,NN
umum,JJ
Jenis,NN
dikerahkan,VB
pemkot,NN
besar,JJ
berekor,VB
panjang,JJ
wajah,NN
berwarna,VB
hitam,JJ
Monyet,NN
diikat,VB
dengan,IN
tali,NN
pelatih,NN
mengawasi,VB
mereka,PRP
akan,MD
melepas,VB
begitu,SC
mendekat,VB
Kantor,NNP
berita,NNP
AFP,NNP
melaporkan,VB
stadion,NN
tinju,NN
hockey,FW
mendapatkan,VB
perhatian,NN
khusus,JJ
karena,SC
sering,RB
diserbu,VB
dua,CD
serbuan

atau,CC
juru bicara,NN
menambahkan,VB
pelamar,NN
16,CD
siapa pun,SC
ketentuan,NN
ditolak,VB
',Z
Berkaki,VB
pendek,JJ
Meski,SC
syarat,NN
layak,JJ
dipilih,VB
jangkung,JJ
antara,NN
karakter,NN
fiksi,NN
ciptaan,NN
JRR,NNP
Tolkien,NNP
Dalam,IN
prolog,NN
Of,NNP
menggambarkan,VB
Hobbits,NNP
berukuran,VB
hingga,IN
tiga,CD
kaki,NND
sekitar,IN
120,CD
rata-rata,NN
inci,NND
75,CD
Surat-surat,NN
penampakan,NN
fisik,NN
sosok,NN
protagonis,NN
Bilbo,NNP
Baggins,NNP
agak,RB
tambun,JJ
perut,NN
kaki,NN
Wajah,NN
bulat,JJ
jenaka,JJ
;,Z
telinga,NN
sedikit,RB
runcing,JJ
kurcaci,NN
rambut,NN
keriting,JJ
cokelat,JJ
Kaki,NN
mata kaki,NN
bawah,NN
tertutup,VB
bulu,NN
tambah,VB
Pencarian,NN
casting,FW
dijadwalkan,VB
hari,NN
National,NNP
Dance,NNP
and,NNP
Drama,NNP
Centre,NNP
Sutradara,NN
kelahiran,NN
Meksiko,NNP
Guillermo,NNP
Del,NNP
Toro,NNP
bersedia,VB
sutradara,NN
tapi,CC
menarik diri,VB
alasan,NN
produksi,NN
tertunda,VB
terbaik,JJ
menggantikan,VB
meski,SC
pengumuman,NN
pergantian,NN
Pencurian,NN
ladang,NN
angg

Taiwan,NNP
0.8,CD
sawit,NN
kuartal,NN
TBS,NN
tandan,NN
buah,NN
segar,JJ
I,OD
808.475,CD
Penurunan,NN
akibat,NN
musim kering,NN
semester,NN
II,OD
dirasakan,VB
Julie,NNP
Syaftari,NNP
Kamis,NNP
tersebar,VB
Misal,NN
Kalimantan,NNP
Sulawesi,NNP
berakibat,VB
mentah,JJ
223,CD
200.662,CD
Lembaga,NNP
Penjamin,NNP
Simpanan,NNP
LPS,NNP
menurunkan,VB
penjaminan,NN
rate,FW
14,CD
25,CD
basis,NND
poin,NND
bps,NND
simpanan,NN
bank umum,NN
Bank,NN
Perkreditan,NN
Rakyat,NN
BPR,NN
Makro,NN
nilai tukar,NN
stabil,JJ
Eksekutif,NN
Krisna,NNP
Wijaya,NNP
mulai,VB
memberlakukan,VB
batas,NN
dijamin,VB
kepercayaan,NN
level,FW
of,FW
confidence,FW
bentuk,NN
dipertahankan,VB
level,NN
maka,SC
BI,NNP
kembali,VB
Rapat,NN
Dewan,NN
Gubernur,NN
RDG,NN
mempertahankan,VB
Lembaga,NN
pemeringkat,NN
internasional,JJ
Fitch,NNP
Ratings,NNP
peringkat,NN
AA,SYM
idn,SYM
minus,JJ
Pan,NNP
Panin,NNP
berdomisili,VB
siaran pers,NN
diterima,VB
ANTARA,NNP
menaikkan,VB
dukungan,NN
Support,FW
Rating,FW
menegaskan,VB
individu,NN
Individual,F

Rawamangun,NNP
Lebih,RB
batang,NND
ditujukan,VB
gap,NN
ketimpangan,NN
golongan,NN
Ketimpangan,NN
antar,IN
cenderung,JJ
mengenalkan,VB
Karena,IN
infrastruktur,NN
pendidikan,NN
mendadak,RB
Keputusan,NN
diambil,VB
supaya,SC
kepastian hukum,NN
pelaku,NN
sinyal,NN
sementara,CC
diberlakukan,VB
pemeringkat,NNP
internasional,NNP
Ratings,FW
merevisi,VB
outlook,FW
mata uang,NN
Issuer,FW
Default,FW
IDRs,FW
BB-,SYM
BB,SYM
IDR,SYM
jangka pendek,NN
B,SYM
Ceiling,FW
Revisi,NN
Sovereign,FW
disiplin,NN
menyeluruh,JJ
melaksanakan,VB
struktural,JJ
bertujuan,VB
Ai,NNP
Ling,NNP
Ngiam,NNP
Direktur,NN
Tim,NNP
Sovereign,NNP
statemen,X
rating,FW
birokrasi,NN
perlahan,JJ
Budaya,NN
takut,JJ
terbuka,JJ
seraya,IN
investigasi-investigasi,NN
antikorupsi,JJ
mengarah,VB
tuntutan,NN
berprofil,VB
pemulihan,NN
pengaduan,NN
tender,NN
elektronis,JJ
transparansi,NN
Daya,NNP
Saing,NNP
Global,NNP
World,NNP
Economic,NNP
69,CD
Disebutkan,VB
luas,JJ
kekuatan,NN
memproyeksikan,VB
utang,NN
38,CD
1997,CD
median,NN
41,CD
212,CD
162,

124.45,CD
257.145,CD
dipasarkan,VB
Vietnam,NNP
695.517,CD
761.253,CD
5.763,CD
Dolar,NN
diperdagangkan,VB
119,CD
yen,NN
tokyo,NNP
datar,JJ
pukul,NN
57,CD
Tokyo,NNP
Euro,NN
2941,CD
71,CD
2942,CD
2994,CD
tadi,NN
menyusutnya,NN
turunnya,NN
memicu,VB
Federal,NNP
Reserve,NNP
ulang,RB
dealer,FW
Hak,NN
Memesan,VB
Efek,NN
Terlebih,RB
Dahulu,NN
Persiapan,NN
action,FW
pengkajian,NN
Abdams,NNP
Katoppo,NNP
berkenaan,VB
divestasi,NN
19,CD
dinyatakan,VB
privatisasi,NN
Fee,FW
based,FW
Wadirut,NNP
Wayan,NNP
Mertayasa,NNP
Uni,NNP
Emirat,NNP
Arab,NNP
UEA,NNP
Sheikh,NNP
Khalifa,NNP
bin,NNP
Zayed,NNP
Al,NNP
Nahyan,NNP
mendirikan,VB
Islam,NNP
Hilal,NNP
Bulan,NNP
Sabit,NNP
dirham,NN
syariat,NN
modern,JJ
real,FW
estate,FW
Investasi,NN
Abu,NNP
Dhabi,NNP
tunggal,JJ
menetapkan,VB
Gulf,NNP
News,NNP
diputuskan,VB
Eisa,NNP
Mohammad,NNP
Suwaidi,NNP
Anggota,NN
Hassan,NNP
Omran,NNP
Nasser,NNP
Yunus,NNP
Haji,NNP
Khoory,NNP
Ahmad,NNP
Sari,NNP
Mazroui,NNP
syariah,JJ
menginvestasikan,VB
kerangka,NN
syariah,X
senior,JJ
per

In [128]:
def viterbi(trans_prob, emission_prob, tag_count, sentence):
    #initialization
    viterbi_mat = {}
    tag_sequence = []
    
    sentence_words = sentence.split()
    for i in range(0,len(sentence_words)):
        
    
    return viterbi_mat, tag_sequence

IndentationError: expected an indented block (<ipython-input-128-ff1fcbe89513>, line 10)